In [5]:
# Imports:
import pandas as pd
import numpy as np
from methods import *
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier


train = r'data\train.csv'
test = r'data\test.csv'

In [2]:
#load the training set
data = pd.read_csv(train)

# In the real world, name should not have any impact on the survival of any given person. For this reason, and because each passenger has a unique ID, we will be omitting the names from the dataset.
data = data.drop("Name", axis = 1)

#fill empty values with large negative value (-inf)
data = data.fillna(-10000000000)

#hash alphanumerical data
data["Ticket"] = data["Ticket"].apply(lambda x: int(hash(x)))
data["Cabin"] = data["Cabin"].apply(lambda x: int(hash(x)))

# change categorical data to integers
data["Sex"] = data["Sex"].apply(lambda x: 1 if x == 'male' else 0)
data["Embarked"] = data["Embarked"].apply(lambda x: condit(x))

#convert the entries to floats
strToFloat(data)

#normalize data
normData(data)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1.0,0.0,0.695702,0.676401,0.621899,0.606483,0.383810,0.154507,0.377032,0.495648,0.356854
1,2.0,1.0,0.172898,0.205085,0.621899,0.606483,0.383810,0.247688,0.687059,0.651857,0.873046
2,3.0,1.0,0.695702,0.205085,0.621899,0.383604,0.383810,0.219106,0.380228,0.495648,0.356854
3,4.0,1.0,0.172898,0.205085,0.621899,0.606483,0.383810,0.589161,0.603601,0.094597,0.356854
4,5.0,0.0,0.695702,0.676401,0.621899,0.383604,0.383810,0.375371,0.380821,0.495648,0.356854
...,...,...,...,...,...,...,...,...,...,...,...
886,887.0,0.0,0.408745,0.676401,0.621899,0.383604,0.383810,0.316492,0.404571,0.495648,0.356854
887,888.0,1.0,0.172898,0.205085,0.621899,0.383604,0.383810,0.474560,0.488913,0.046632,0.356854
888,889.0,0.0,0.695702,0.205085,0.118436,0.606483,0.881614,0.391832,0.456072,0.495648,0.356854
889,890.0,1.0,0.172898,0.676401,0.621899,0.383604,0.383810,0.172273,0.488913,0.827385,0.873046


In [3]:
model = DecisionTreeClassifier()

x_train = data.drop("Survived", axis = 1).drop("PassengerId", axis = 1)
y_train = data["Survived"]

model.fit(x_train,y_train)

DecisionTreeClassifier()

In [4]:
#Similar process for the test set except now we are predicting the "Survived" status
dataTest = pd.read_csv(test)

dataTest = dataTest.drop("Name", axis = 1)
dataTest = dataTest.fillna(-10000000000)
dataTest["Ticket"] = dataTest["Ticket"].apply(lambda x: int(hash(x)))
dataTest["Cabin"] = dataTest["Cabin"].apply(lambda x: int(hash(x)))
dataTest["Sex"] = dataTest["Sex"].apply(lambda x: 1 if x == 'male' else 0)
dataTest["Embarked"] = dataTest["Embarked"].apply(lambda x: condit(x))

strToFloat(dataTest)

#drop "PassengerId"
x_test = normData(dataTest).drop("PassengerId", axis = 1)

# predict using the moodel
pred = model.predict(x_test)

# output to .csv file
out = pd.DataFrame({"PassengerId" : dataTest["PassengerId"],"Survived" : pred})
print(out)
#out.to_csv('data\submission.csv', index = False)

     PassengerId  Survived
0          892.0       1.0
1          893.0       1.0
2          894.0       0.0
3          895.0       1.0
4          896.0       0.0
..           ...       ...
413       1305.0       0.0
414       1306.0       1.0
415       1307.0       0.0
416       1308.0       0.0
417       1309.0       0.0

[418 rows x 2 columns]


After rerunning the decision tree model it seems that our output changes frequently which would imply that the model is not a good fit. Lets try something else. Since we are looking to classify, and not looking at the probability that a given passenger survived, we will be using the Random Forest Classifier Algorithm

In [55]:
model = RandomForestClassifier(n_estimators=1000)

model.fit(x_train,y_train)

pred = model.predict(x_test)
out = pd.DataFrame({"PassengerId" : dataTest["PassengerId"],"Survived" : pred})

# seems to be far more consistent with output
print(out)
out.to_csv('data\submission.csv', index = False)

     PassengerId  Survived
0          892.0       0.0
1          893.0       0.0
2          894.0       0.0
3          895.0       0.0
4          896.0       0.0
..           ...       ...
413       1305.0       0.0
414       1306.0       1.0
415       1307.0       0.0
416       1308.0       0.0
417       1309.0       0.0

[418 rows x 2 columns]
